In [6]:
import numpy as np
from scipy.linalg import block_diag
from cvxopt import matrix, solvers ,glpk

m=np.random.randint(10,15)
n=np.random.randint(10,15)
#create sparse matrix
mat=np.random.randint(15,40,(m,n))
p=mat[:-1,:]
p[p%25 -1 !=0]=0

#remove void rows and columns
rs=np.sum(p,axis=1)
cs=np.sum(p,axis=0)
rows=np.where(rs!=0)[0]
rows=np.append(rows,m-1)
cols=np.where(cs!=0)[0]
mat=mat[:,cols][rows,:]
    
#create supply demand
right=mat.sum(axis=1)
down=mat.sum(axis=0)
sparse=np.copy(mat)
sparse[sparse>0]=1


sparse="""0	0	0	1	0	1	0	1
0	0	0	1	1	1	1	1
0	0	1	0	1	1	0	0
1	0	0	0	0	0	0	0
1	1	1	0	0	0	0	0
1	1	1	1	1	1	1	1"""
sparse=np.array([list(map(int, k.split("\t"))) for k in sparse.split("\n")])

down=[92,32,93,83,78,138,55,75]
right=[96,149,90,28,81,202]

# sparse=np.array([[1, 0,1],
#        [0, 1,1],
#        [1, 1,1]])
# down=[40,40,40]
# right=[30,40,50]

def get_constraints(sparse_,down_,right_):
    row_constrinats=block_diag(*sparse_)
    column_constraints=np.concatenate(np.apply_along_axis(lambda x: block_diag(*x),1,sparse_)).T
    A=np.append(row_constrinats,column_constraints,axis=0)
    A=A.astype("d")
    rows=np.where(np.sum(A,axis=1)!=0)[0]
    cols=np.where(np.sum(A,axis=0)!=0)[0]
    A=A[:,cols][rows,:]
    variables=A.shape[1]
    b=np.concatenate([right_, down_], axis=0) 
    b=b.astype("d")
    c=np.ones(variables)
    rank=np.linalg.matrix_rank(A)
    #A=A[:rank,:]
    #b=b[:rank]
    g=-1*np.identity(variables,dtype='d')
    H=np.zeros(variables,dtype='d')
    return A,g,b,H


In [2]:
A,G,b,h=get_constraints(sparse,down,right)

In [3]:
variables=list(zip(np.where(A==1)[0],np.where(A==1)[1]))
A1 = np.copy(A)
A1 = np.append(A1,np.matrix(b).T, axis = 1)

G1 = np.copy(G)
G1 = np.append(G1,np.matrix(h).T, axis = 1)
for col in range(A.shape[1]-sparse.shape[1]):
    print("updating column: " + str(col))
    current_variable_coulmn_A=A1[:,col]
    equality_constraint_rows=np.where(current_variable_coulmn_A!=0)[0]
    current_variable_coulmn_G=G1[:,col]
    positive_inequality_rows=np.where(current_variable_coulmn_G==1)[0]
    negative_inequality_rows=np.where(current_variable_coulmn_G==-1)[0]
    inequality_constraint_rows = np.append(positive_inequality_rows, negative_inequality_rows)
    print(equality_constraint_rows.shape[0])
    if equality_constraint_rows.shape[0]==2:
        for current_inequality_row in inequality_constraint_rows:
            # Optimise if-else using radamacher 
            if G1[current_inequality_row, col] == A1[equality_constraint_rows[1], col]:
                G1[current_inequality_row, :] = G1[current_inequality_row, :] - A1[equality_constraint_rows[1], :]
            else:
                G1[current_inequality_row, :] = G1[current_inequality_row, :] + A1[equality_constraint_rows[1], :]
        
        if A1[equality_constraint_rows[0],col] == A1[equality_constraint_rows[1],col]:
            A1[equality_constraint_rows[0],:] = A1[equality_constraint_rows[0],:] - A1[equality_constraint_rows[1],:]
        else:
            A1[equality_constraint_rows[0],:] = A1[equality_constraint_rows[0],:] + A1[equality_constraint_rows[1],:]
        A1=np.delete(A1, equality_constraint_rows[1], 0)

    else:
        cross=[(i,j) for i in positive_inequality_rows for j in negative_inequality_rows]
        for (p, n) in cross:
            G1 = np.append(G1, (G1[p,:] + G1[n,:]), axis = 0)
        G1 = np.delete(G1, inequality_constraint_rows, 0)
            

updating column: 0
2
updating column: 1
2
updating column: 2
2
updating column: 3
2
updating column: 4
2
updating column: 5
0
updating column: 6
2
updating column: 7
0
updating column: 8
2
updating column: 9
2
updating column: 10
0
updating column: 11
2
updating column: 12
2
updating column: 13
2
updating column: 14
2


In [4]:
A1

matrix([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,   -1.,   -1.,   -1.,
           -1.,   -1.,   -1.,   -1.,   -1., -202.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    1.,    1.,    1.,
            1.,    1.,    1.,    1.,    1.,  202.]])

In [5]:
G1

matrix([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    1.,    0.,   55.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    1.,    1.,    1.,
            0.,    0.,    0.,    0.,    0.,  108.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,   28.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,   64.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    0.,
            0.,    0.,    0.,    0.,    0.,   32.],
        [   0.,    0

In [221]:
sparse

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])